In [1]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [2]:
#del sys.modules["utilities"]
from utilities import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [5]:
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))


feats_n2v_all_train = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_train.pkl','rb'))
feats_n2v_all_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_val.pkl','rb'))
feats_n2v_all_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_test.pkl','rb'))



Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels.pkl','rb'))

In [6]:
cutoff_y = np.quantile(Outcome_labels['LOS_postoperative'],0.75)
cutoff_y
Outcome_labels['PLOS'] =  (Outcome_labels['LOS_postoperative']>cutoff_y)*1
del Outcome_labels['LOS_postoperative']

Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [7]:
df_feats_clinical_n2v_train = df_feats_clinical[df_feats_clinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                    merge(feats_n2v_all_train,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_n2v_val = df_feats_clinical[df_feats_clinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                    merge(feats_n2v_all_val,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_n2v_test = df_feats_clinical[df_feats_clinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(feats_n2v_all_test,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


len(df_feats_clinical_n2v_train)+len(df_feats_clinical_n2v_val)+len(df_feats_clinical_n2v_test)-len(df_feats_clinical)

0

In [8]:
X_train = df_feats_clinical_n2v_train
X_val = df_feats_clinical_n2v_val
X_test = df_feats_clinical_n2v_test

y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_test = Outcome_labels_test

In [9]:
X_train_val = pd.concat([X_train,X_val])
y_train_val = pd.concat([y_train,y_val])

In [10]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 3017),
 (13665, 3017),
 (122983, 3017),
 (13664, 3017),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [11]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [12]:
del sys.modules["utilities"]
from utilities import *

In [13]:
experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[666])

100%|████████████████████████████████████████| 1/1 [3:56:18<00:00, 14178.36s/it]


A. Scores on training set
avg XGBoost classifer: auc/  0.913  F1:  0.6656  precision:  0.8057  recall:  0.5671  accuracy:  0.86
std XGBoost classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg MLP classifer: auc/  0.8359  F1:  0.5452  precision:  0.6664  recall:  0.4612  accuracy:  0.8109
std MLP classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg logistic classifer: auc/  0.8489  F1:  0.618  precision:  0.5212  recall:  0.7592  accuracy:  0.7695
std logistic classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
****************************************************

B. Scores on test set
avg XGBoost classifer: auc/  0.8448  F1:  0.5724  precision:  0.7197  recall:  0.4752  accuracy:  0.7951
std XGBoost classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg MLP classifer: auc/  0.8258  F1:  0.566  precision:  0.6855  recall:  0.482  accuracy:  0.7867
std MLP classifer: auc/  0.0 